# Transfer Learning with GPT-2 [Modeling]
### Data Mining Final Project
(Working through tutorials) - https://towardsdatascience.com/how-to-fine-tune-gpt-2-for-text-generation-ae2ea53bc272<br>

Khyatee Desai<br>
kdesai1@gradcenter.cuny.edu<br>
December 2022

In [4]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer,AdamW, get_linear_schedule_with_warmup
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv

In [5]:
### Prepare data
lyrics = pd.read_csv('lyrics-data.csv')
lyrics = lyrics[lyrics['language']=='en']
lyrics

,ALink,SName,SLink,Lyric,language
69,/ivete-sangalo/,Careless Whisper,/ivete-sangalo/careless-whisper.html,I feel so unsure\nAs I take your hand and lead...,en
86,/ivete-sangalo/,Could You Be Loved / Citação Musical do Rap: S...,/ivete-sangalo/could-you-be-loved-citacao-musi...,"Don't let them fool, ya\nOr even try to school...",en
88,/ivete-sangalo/,Cruisin' (Part. Saulo),/ivete-sangalo/cruisin-part-saulo.html,"Baby, let's cruise, away from here\nDon't be c...",en
111,/ivete-sangalo/,Easy,/ivete-sangalo/easy.html,"Know it sounds funny\nBut, I just can't stand ...",en
140,/ivete-sangalo/,For Your Babies (The Voice cover),/ivete-sangalo/for-your-babies-the-voice-cover...,You've got that look again\nThe one I hoped I ...,en
...,...,...,...,...,...
379926,/clegg-johnny/,The Waiting,/clegg-johnny/the-waiting.html,Chorus\nHere we stand waiting on the plain\nDa...,en
379927,/clegg-johnny/,Too Early For The Sky,/clegg-johnny/too-early-for-the-sky.html,I nearly disappeared into the mouth of a croco...,en
379928,/clegg-johnny/,Warsaw 1943 (I Never Betrayed The Revolution),/clegg-johnny/warsaw-1943-i-never-betrayed-the...,"Amambuka, amambuka azothengisa izwe lakithi, i...",en
379929,/clegg-johnny/,When The System Has Fallen,/clegg-johnny/when-the-system-has-fallen.html,Sweat in the heat for days on end\nwaiting for...,en


In [21]:
#Only keep popular artists, with genre Rock/Pop and popularity high enough
artists = pd.read_csv('artists-data.csv')
names = artists[(artists['Genres'].isin(['Rock']))]['Artist']
artists = artists[(artists.Artist.isin(names)) & artists.Popularity>0]
df = lyrics.merge(artists[['Artist', 'Genres', 'Link']], left_on='ALink', right_on='Link', how='inner')
df = df.drop(columns=['ALink','SLink','language','Link'])
df

,SName,Lyric,Artist,Genres
0,What's Up,Twenty-five years and my life is still\nTrying...,4 Non Blondes,Rock
1,Spaceman,Starry night bring me down\nTill I realize the...,4 Non Blondes,Rock
2,Pleasantly Blue,Every time you wake in the mornin'\nAnd you st...,4 Non Blondes,Rock
3,I'm The One,Ah-hah!\nWoo!\nAh-ha-ha-ha-ha-ha!\nWe came her...,4 Non Blondes,Rock
4,Dear Mr. President,I'm looking outside of my windows\nThe view th...,4 Non Blondes,Rock
...,...,...,...,...
2518,Three Button Hand Me Down,"(Ian McLagan, Rod Stewart)\n\nI don't need no ...",Faces,Rock
2519,Too Bad,"(Ron Wood, Rod Stewart)\n\nToo bad we were thr...",Faces,Rock
2520,Wicked Messenger,(Bob Dylan)\n\nThere was a wicked messenger\nf...,Faces,Rock
2521,"You Can Make Me Dance, Sing, Or Anything","(Jones, McLagan, Stewart, Wood, Yamauchi)\n\nh...",Faces,Rock


In [22]:
#Drop the songs with lyrics too long (after more than 1024 tokens, does not work)
df = df[df['Lyric'].apply(lambda x: len(x.split(' ')) < 350)]

#Create a very small test set to compare generated text with the reality
test_set = df.sample(n = 200)
df = df.loc[~df.index.isin(test_set.index)]

#Reset the indexes
test_set = test_set.reset_index()
df = df.reset_index()

#For the test set only, keep last 20 words in a new column, then remove them from original column
test_set['True_end_lyrics'] = test_set['Lyric'].str.split().str[-20:].apply(' '.join)
test_set['Lyric'] = test_set['Lyric'].str.split().str[:-20].apply(' '.join)

In [8]:
class SongLyrics(Dataset):  
    def __init__(self, control_code, truncate=False, gpt2_type="gpt2", max_length=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.lyrics = []

        for row in df['Lyric']:
          self.lyrics.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))               
        if truncate:
            self.lyrics = self.lyrics[:20000]
        self.lyrics_count = len(self.lyrics)
        
    def __len__(self):
        return self.lyrics_count

    def __getitem__(self, item):
        return self.lyrics[item]
    
dataset = SongLyrics(df['Lyric'], truncate=True, gpt2_type="gpt2")      

In [9]:
#Get the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [10]:
def train(
    dataset, model, tokenizer,
    batch_size=16, epochs=5, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):
    acc_steps = 100
    device=torch.device("cuda")
    model = model.cuda()
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [11]:
model = train(dataset, model, tokenizer)

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
0it [00:00, ?it/s]

Training epoch 0
0


2242it [03:05, 12.09it/s]
0it [00:00, ?it/s]

Training epoch 1
tensor(1.9169, device='cuda:0', grad_fn=<NllLossBackward0>)


2242it [03:10, 11.79it/s]
0it [00:00, ?it/s]

...

2242it [03:07, 11.98it/s]
0it [00:00, ?it/s]

Training epoch 3
tensor(1.0327, device='cuda:0', grad_fn=<NllLossBackward0>)


2242it [02:59, 12.49it/s]
0it [00:00, ?it/s]

Training epoch 4
tensor(1.1394, device='cuda:0', grad_fn=<NllLossBackward0>)


2242it [03:03, 12.25it/s]


In [24]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=30, #maximum number of words
    top_p=0.8,
    temperature=1.):
    
    model.eval()
    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
                output_list = list(generated.squeeze().numpy())
                output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
                generated_list.append(output_text)
                
    return generated_list

#Function to generate multiple sentences. Test data should be a dataframe
def text_generation(test_data):
    generated_lyrics = []
    for i in range(len(test_data)):
        x = generate(model.to('cpu'), tokenizer, test_data['Lyric'][i], entry_count=1)
        generated_lyrics.append(x)
    return generated_lyrics

#Run the functions to generate the lyrics
generated_lyrics = text_generation(test_set)

100%|██████████| 1/1 [00:13<00:00, 13.27s/it]


In [33]:
#Loop to keep only generated text and add it as a new column in the dataframe
my_generations=[]

for i in range(len(generated_lyrics)):
    a = test_set['Lyric'][i].split()[-30:] #Get the matching string we want (30 words)
    b = ' '.join(a)
    c = ' '.join(generated_lyrics[i]) #Get all that comes after the matching string
    my_generations.append(c.split(b)[-1])

test_set['Generated_lyrics'] = my_generations


#Finish the sentences when there is a point, remove after that
# final=[]

# for i in range(len(test_set)):
#     to_remove = test_set['Generated_lyrics'][i].split('.')[-1]
#     final.append(test_set['Generated_lyrics'][i].replace(to_remove,''))

# test_set['Generated_lyrics'] = final

In [45]:
test_set

,index,SName,Lyric,Artist,Genres,True_end_lyrics,Generated_lyrics
0,1349,One With You,Angel I want to,Santana,Rock,be one with you Spread your wings and let me i...,hear what you think about this article. Submi...
1,1978,Made a Mess of Me,Yeah I felt so free Life's a breeze I didn't s...,Stereophonics,Rock,Yeah You turned me over You stitched me up And...,In California Was bittersweet And in the morn...
2,799,Say Sons,"Well, I got a little girl down by the river No...",Bruce Springsteen,Rock,"on, down by the river oh, come on, girl, it's ...","Well, I got a little girl down by the river No..."
3,774,Ramrod,"Hey, little dolly with the blue jeans on I wan...",Bruce Springsteen,Rock,little girl I'll put my foot to the floor Give...,down here in a few days I'll be there tonight...
4,1743,Fault Lines,See those fault lines Laid out like land mines...,Tom Petty,Rock,of my own I've got a few of my own fault lines...,of my own I've got a few of my own fault line...
...,...,...,...,...,...,...,...
195,1840,The Golden Rose,"Well, The Golden Rose sailed with a broken man...",Tom Petty,Rock,Many a-night I would think of her All alone Ch...,I was gone\n\n\nNow I've told you stories and...
196,1150,I Believe To My Soul,One of these days and it won't be long Youre g...,The Animals,Rock,"Oh yes, I believe I believe to my soul You're ...",I believe to my soul you're tryin' to beat me...
197,1114,Many Rivers To Cross,Eric Burdon & Animals - Many Rivers To Cross M...,The Animals,Rock,"say why Well I guess, I have to try, yeah yeah...","say why Well I guess, I have to try Many rive..."
198,134,Even A Fool Learns To Love,"There was a time, the laughing time I took my ...",David Bowie,Rock,On a clown and an angel so much in love I'll s...,On a clown and an angel so much in love I'll ...


In [ ]:
#Using BLEU score to compare the real sentences with the generated ones
import statistics
from nltk.translate.bleu_score import sentence_bleu

scores=[]

for i in range(len(test_set)):
    reference = test_set['True_end_lyrics'][i]
    candidate = test_set['Generated_lyrics'][i]
    scores.append(sentence_bleu(reference, candidate))

statistics.mean(scores)